In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e12/sample_submission.csv
/kaggle/input/playground-series-s3e12/train.csv
/kaggle/input/playground-series-s3e12/test.csv


In [2]:
train_data=pd.read_csv('/kaggle/input/playground-series-s3e12/train.csv')
test_data=pd.read_csv('/kaggle/input/playground-series-s3e12/test.csv')

In [3]:
train_data.head()

,id,gravity,ph,osmo,cond,urea,calc,target
0,0,1.013,6.19,443,14.8,124,1.45,0
1,1,1.025,5.40,703,23.6,394,4.18,0
2,2,1.009,6.13,371,24.5,159,9.04,0
3,3,1.021,4.91,442,20.8,398,6.63,1
4,4,1.021,5.53,874,17.8,385,2.21,1


In [4]:
train_data.isna().sum()

id         0
gravity    0
ph         0
osmo       0
cond       0
urea       0
calc       0
target     0
dtype: int64

In [5]:
(train_data == ' ').sum()

id         0
gravity    0
ph         0
osmo       0
cond       0
urea       0
calc       0
target     0
dtype: int64

In [6]:
X=train_data.drop(['id','target'],axis=1)
Y=train_data['target']

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
sc.fit(X_test)
X_test_std = sc.transform(X_test)

In [9]:
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors=5, 
                           p=2, 
                           metric='minkowski')
dt = DecisionTreeClassifier(criterion='gini', 
                                    max_depth=4, 
                                    random_state=1)
svm = SVC(kernel='linear', C=1.0, random_state=1)
svmrbf = svm = SVC(kernel='rbf', random_state=1, gamma=0.1, C=1.0)
randomforestmodel = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
lrmodel = LogisticRegression(C=100.0, solver='liblinear', multi_class='ovr')
ppn = Perceptron(eta0=0.1, random_state=1)

In [10]:
ppn.fit(X_train_std, y_train)

Perceptron(eta0=0.1, random_state=1)

In [11]:
svmrbf.fit(X_train_std, y_train)

SVC(gamma=0.1, random_state=1)

In [12]:
knn.fit(X_train_std, y_train)

KNeighborsClassifier()

In [13]:
svm.fit(X_train_std, y_train)

SVC(gamma=0.1, random_state=1)

In [14]:
dt.fit(X_train_std, y_train)

DecisionTreeClassifier(max_depth=4, random_state=1)

In [15]:
lrmodel.fit(X_train_std, y_train)

LogisticRegression(C=100.0, multi_class='ovr', solver='liblinear')

In [16]:
randomforestmodel.fit(X_train_std, y_train)

RandomForestClassifier(max_depth=5, random_state=1)

In [17]:
y_pred_ppn= ppn.predict(X_test_std)
y_pred_svmrbf= svmrbf.predict(X_test_std)
y_pred_knn= knn.predict(X_test_std)
y_pred_dt= dt.predict(X_test_std)
y_pred_svm=svm.predict(X_test_std)
y_pred_lr=lrmodel.predict(X_test_std)
y_pred_rf=randomforestmodel.predict(X_test_std)

In [18]:
accuracyppn= accuracy_score(y_test, y_pred_ppn)
accuracylr= accuracy_score(y_test, y_pred_lr)
accuracyrf= accuracy_score(y_test, y_pred_rf)
accuracydt= accuracy_score(y_test, y_pred_dt)
accuracyknn= accuracy_score(y_test, y_pred_knn)
accuracysvmrbf= accuracy_score(y_test, y_pred_svmrbf)
accuracysvm= accuracy_score(y_test, y_pred_svm)
print("Accuracy of svm:",accuracysvm)
print("Accuracy of svmrbf:",accuracysvmrbf)
print("Accuracy of knn:",accuracyknn)
print("Accuracy of dt:",accuracydt)
print("Accuracy of rf:",accuracyrf)
print("Accuracy of lr:",accuracylr)
print("Accuracy of ppn:",accuracyppn)

Accuracy of svm: 0.7349397590361446
Accuracy of svmrbf: 0.7349397590361446
Accuracy of knn: 0.6626506024096386
Accuracy of dt: 0.7831325301204819
Accuracy of rf: 0.7710843373493976
Accuracy of lr: 0.7710843373493976
Accuracy of ppn: 0.7831325301204819


In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
pn=Perceptron(random_state=1)
pipe = Pipeline(steps=[("sc", sc),("pn", pn)])

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'pn__eta0': param_range}]

gs = GridSearchCV(estimator=pipe, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=10)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.6523172905525847
{'pn__eta0': 0.01}


In [20]:
ppn2 = Perceptron(eta0=0.01, random_state=1)

In [21]:
ppn2.fit(X_train_std, y_train)

Perceptron(eta0=0.01, random_state=1)

In [22]:
y_pred_ppn2= ppn2.predict(X_test_std)

In [23]:
accuracyppn2= accuracy_score(y_test, y_pred_ppn2)
print("Accuracy of ppn2:",accuracyppn2)

Accuracy of ppn2: 0.7831325301204819


In [24]:
test_data.head()

,id,gravity,ph,osmo,cond,urea,calc
0,414,1.017,5.24,345,11.5,152,1.16
1,415,1.020,5.68,874,29.0,385,3.46
2,416,1.024,5.36,698,19.5,354,13.00
3,417,1.020,5.33,668,25.3,252,3.46
4,418,1.011,5.87,567,29.0,457,2.36


In [25]:
X_test_data= test_data.drop(['id'], axis=1)

In [26]:
X_test_data.isna().sum()

gravity    0
ph         0
osmo       0
cond       0
urea       0
calc       0
dtype: int64

In [27]:
(X_test_data == ' ').sum()

gravity    0
ph         0
osmo       0
cond       0
urea       0
calc       0
dtype: int64

In [28]:
sc.fit(X_test_data)
X_test_data_scaled= sc.transform(X_test_data)

In [29]:
predictionsppn = ppn2.predict(X_test_data_scaled)

In [30]:
submission_datappn = pd.DataFrame({'ID': test_data['id'], 'target': predictionsppn})
submission_datappn.to_csv('submission.csv', index=False)